# Memory Networks

So far, we have covered two components that have enable neural network models to solve textual tasks: **word embeddings** and **attention**. In this notebook, we will be introducing a **memory** component, which is relevant to cases in which long-term memory is needed, such as answering questions about a sequence of events.

While RNNs do have a memory component in the form of a hidden state, this often does not sufficiently capture long-term dependencies, as such long-term information must be condensed into a single dense vector representation. Memory networks were designed to overcome this information bottleneck.

In this notebook, we will build the base model for an end-to-end trainable memory network (["End-To-End Memory Networks" (Sukhbaatar et al.)](https://arxiv.org/pdf/1503.08895.pdf].)).

In [1]:
%matplotlib inline

import collections
from functools import partial
import math
import matplotlib.pyplot as plt
import os
import random
import time
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange

import tensorflow as tf

# Helper TensorFlow functions
from utils import get_session, maybe_download

/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Data
You will evaluate your results on Facebook's bAbi dataset, which assesses performance on 20 different question answering tasks for reasoning over text.

In [2]:
# Get Facebook's bAbi dataset
from memn2n.babi_utils import get_babi_en

#get_babi_en()

# For 10K dataset, uncomment below:
get_babi_en(get_10k=True)

Finished!
Found and verified datasets/babi_tasks_1-20_v1-2.tar.gz
Some housekeeping...
Finished.


# MemN2N

The base MemN2N model has the following components:

## Input Map

First, we will need to convert our data, which come in the form of **stories** (a list of facts, typically a description of events; e.g.,
*joe go playground; bob go office; joe get football*), the **query** being asked about our story (e.g. *where is joe?*) and its respective **answer** into an internal feature representation.

Here, we used an input map that assigns a unique ID to each word in the vocabulary of the stories and queries (built from words of the story and query of the test and training sets). The answer is one-hot encoded.

As sentences vary in length, we pad sentences with a null symbol so they are padded to the same size. The value of the null embedding was chosen to be 0.

You can find more details about this step in the `get_data_info` function of `memn2n/data_utils.py`.

## Sentence Representation

To represent the positions of words within a sentence, we will adopt positional encodings (PE) to allow the order of words to impact our memories. It takes the form:

$$m_i = \sum_{j}l_j \cdot Ax_{ij}$$

where $\cdot$ is an element wise multiplication, and $l_j$ is a column vector with the following elements:

$$l_{kj} = (1 - j / J) - (k/d)(1 - 2j/J)$$

where $J$ is the number of words in the sentence and $d$ is the dimension of the embedding.

## Q1A. Implement Position Encoding

Open `memn2n/memn2n_skeleton.py` and fill in the `position_encoding` function.

## Input Memory Representation

Our inputs, $x_1,...,x_i$, for stories, queries, and answers need to be stored in memory, represented by memory vectors $m_1,...,m_i$ of dimension $d$. To accomplish this, we will use an embedding matrix A (of size $d \times V$). We will use embedding matrix $B$ to represent the queries into an internal state $u$. 

The match between $u$ and each $m_i$ is computed by taking the inner product and a softmax:

$$p_i = \text{softmax}(u^Tm_i)$$

## Output Memory Representation

Each $x_i$ has a corresponding output vector $c_i$ which is given by another embedding matrix, $C$. The response from the memory $o$ is computed by taking the sum over the transformed inputs $c_i$ weighted by the probability vector from the input:

$$o = \sum_{i}p_ic_i$$

# Generating Predictions

In a single layer memory network, the sum of the output vector $o$ and input embedding $u$ is transformed by a weight matrix $W$ and passed through a softmax to generate a predicted answer.

$$\hat{a} = \text{softmax}(W(o + u))$$

To extend this to a multiple layer model, we will iterate over the memory $K$ times, or for $K$ **hops**, and adopt **adjacent weight tying.**

The layers are stacked as such:

* The question embedding is constrained to match the input embedding of the first layer: 
$$B = A^1$$


* The output embedding for one layer is the input embedding for the layer above it: 
$$A^{k+1}= C^k$$


* The inputs to all layers following the first is the sum of the output $o^k$ and the input $u^k$ from layer $k$: $$u^{k+1}=u^k+o^k$$


* The input to W combines the input and output of the top memory layer: 
$$\hat{a} = \text{softmax}(W(o^k + u^k))$$


* The answer prediction matrix is constrained to be the same as the final output embedding: 
$$W^T = C^K$$

<figure>
    <img src='images/arch.png' alt='missing' />
    <figcaption>**Figure 1.** (a):  A single layer version of our model. (b):  A three layer version of our model. In practice, we will constrain several of the embedding matrices to be the same.
    </figcaption>
</figure>

# Q2, Q3, Q4

Implement the MemN2N model according to the above specifications.

In [3]:
from memn2n.memn2n_skeleton import MemN2N_Base

class MemN2N(MemN2N_Base):
        
    def _build_inputs(self):
        self._stories = tf.placeholder(tf.int32, [None, self._memory_size, self._sentence_size], name="stories")
        self._queries = tf.placeholder(tf.int32, [None, self._sentence_size], name="queries")
        self._answers = tf.placeholder(tf.int32, [None, self._vocab_size], name="answers")
        self._lr = tf.placeholder(tf.float32, [], name="learning_rate")
        
    def _build_vars(self):
        with tf.variable_scope(self._name):
            nil_word_slot = tf.zeros([1, self._embedding_size])
            A = tf.concat(axis=0, values=[ nil_word_slot, self._init([self._vocab_size-1, self._embedding_size]) ])
            C = tf.concat(axis=0, values=[ nil_word_slot, self._init([self._vocab_size-1, self._embedding_size]) ])
            
            self.A_1 = tf.Variable(A, name="A")
            self.C = []

            for hop in range(self._hops):
                with tf.variable_scope('hop_{}'.format(hop)):
                    self.C.append(tf.Variable(C, name="C"))     
        
        self._nil_vars = set([self.A_1.name] + [x.name for x in self.C])
    
    
    def _inference(self, stories, queries):
        with tf.variable_scope("inference"):
            
            
            ##################################################
            ### Q2 - Input memory representation           ###
            ##################################################
            
            q_emb = tf.nn.embedding_lookup(self.A_1, queries) 
            u = [tf.reduce_sum(q_emb * self._encoding, 1)] 
            
            for hop in range(self._hops):
                if hop == 0:
                    input_emb = tf.nn.embedding_lookup(self.A_1, stories) 
                    # Remember to element wise multiply position encoding!
                    
    
                    m_A = tf.reduce_sum(self._encoding*input_emb,2) 
                    
                    
                else:
                    with tf.variable_scope('hop_{}'.format(hop - 1)):
                        input_emb = tf.gather(self.C[hop - 1], stories)
                        m_A = tf.reduce_sum(input_emb*self._encoding, 2)
                        
                        
           
                u_temp = tf.transpose(tf.expand_dims(u[-1], -1), [0, 2, 1])
                dot_prod = tf.reduce_sum(m_A * u_temp, 2)
                probs = tf.nn.softmax(dot_prod)

                ##################################################
                ### Q3 - Output memory representation          ###
                ##################################################
                
                with tf.variable_scope('hop_{}'.format(hop)):
                    out_emb = tf.gather(self.C[hop],stories) 
                # Remember to element wise multiply position encoding!
                m_C = tf.reduce_sum(out_emb*self._encoding, 2)
                tmp_1 = tf.transpose(tf.expand_dims(probs, -1),[0, 2, 1])
                tmp_2 = tf.transpose(m_C,[0,2,1])
                o_k = tf.reduce_sum(tmp_1*tmp_2,2)
                                    

                ##################################################
                ### Q4 - Generating predictions                ###
                ##################################################
                
                u_k = u[-1] + o_k 
                u.append(u_k)
                
            # Final output embedding, W_T = C_K from adjacent weight sharing
            with tf.variable_scope('hop_{}'.format(self._hops)):
                tmp_3 = tf.transpose(self.C[-1],[1,0])
                return tf.nn.softmax(tf.matmul(u_k,tmp_3)) 

# Training:

We will now evaluate your MemN2N implementation on Facebook's bAbi dataset.

In [4]:
from __future__ import absolute_import
from __future__ import print_function

from memn2n.data_utils import get_data_info, split_train_valid_test, generate_batches
from sklearn import metrics
from six.moves import range

import tensorflow as tf
import numpy as np
import pandas as pd

In [5]:
# Parameter intialization

# Hyperparameters
learning_rate = 0.01 # Learning rate for the Adam Optimizer
lr_decay_epoch = 25 # Number of epochs until lr is halved
lr_decay_stop = 100 # Epoch to stop annealing lr
max_grad_norm = 40 # Clip gradients to this norm

hops = 3 # Number of hops
embedding_size = 20 # Embedding size for embedding matrices
memory_size = 50 # Maximum size of memory

epochs = 200
batch_size = 256
evaluation_interval = 10
data_dir = "datasets/babi"

In [6]:
# Get data and process for our model
word_idx, vocab_size, sentence_size = get_data_info(data_dir, memory_size)

train_stories, train_queries, train_answers, \
val_stories, val_queries, val_answers, \
test_stories, test_queries, test_answers = split_train_valid_test(data_dir, word_idx, sentence_size, memory_size)

# Number of train/val/test examples
n_train = train_stories.shape[0]
n_val = val_stories.shape[0]
n_test = test_stories.shape[0]

# Create labels
train_labels = np.argmax(train_answers, axis=1)
test_labels = np.argmax(test_answers, axis=1)
val_labels = np.argmax(val_answers, axis=1)

# Generate batches
batches = generate_batches(batch_size, n_train)

/home/ubuntu/anaconda3/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [ ]:
# Training loop

# Accuracy Results
train_eval = None
val_eval = None
test_eval = None

with tf.Session() as sess:
    model = MemN2N(batch_size, vocab_size, sentence_size, memory_size, embedding_size, session=sess,
                   hops=hops, max_grad_norm=max_grad_norm)
    for i in range(1, epochs):
        
        # Stepped learning rate
        if i - 1 <= lr_decay_stop:
            anneal = 2.0 ** ((i - 1) // lr_decay_epoch)
        else:
            anneal = 2.0 ** (lr_decay_stop // lr_decay_epoch)
        lr = learning_rate / anneal

        np.random.shuffle(batches)
        total_cost = 0.0
        for start, end in batches:
            s = train_stories[start:end]
            q = train_queries[start:end]
            a = train_answers[start:end]
            cost_t = model.batch_fit(s, q, a, lr)
            total_cost += cost_t

        if i % evaluation_interval == 0:
            train_accs = []
            for start in range(0, n_train, n_train//20):
                end = start + n_train//20
                s = train_stories[start:end]
                q = train_queries[start:end]
                pred = model.predict(s, q)
                acc = metrics.accuracy_score(pred, train_labels[start:end])
                train_accs.append(acc)

            val_accs = []
            for start in range(0, n_val, n_val//20):
                end = start + n_val//20
                s = val_stories[start:end]
                q = val_queries[start:end]
                pred = model.predict(s, q)
                acc = metrics.accuracy_score(pred, val_labels[start:end])
                val_accs.append(acc)

            test_accs = []
            for start in range(0, n_test, n_test//20):
                end = start + n_test//20
                s = test_stories[start:end]
                q = test_queries[start:end]
                pred = model.predict(s, q)
                acc = metrics.accuracy_score(pred, test_labels[start:end])
                test_accs.append(acc)

            print('-----------------------')
            print('Epoch', i)
            print('Total Cost:', total_cost)
            print()
            t = 1
            for t1, t2, t3 in zip(train_accs, val_accs, test_accs):
                print("Task {}".format(t))
                print("Training Accuracy = {}".format(t1))
                print("Validation Accuracy = {}".format(t2))
                print("Testing Accuracy = {}".format(t3))
                print()
                t += 1
            print('-----------------------')
        
        if i == epochs:
            train_eval, val_eval, test_eval = train_accs, val_accs, test_accs

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

-----------------------
Epoch 10
Total Cost: 893480.3831787109

Task 1
Training Accuracy = 0.7438888888888889
Validation Accuracy = 0.747
Testing Accuracy = 0.76

Task 2
Training Accuracy = 0.5588888888888889
Validation Accuracy = 0.545
Testing Accuracy = 0.551

Task 3
Training Accuracy = 0.4922222222222222
Validation Accuracy = 0.503
Testing Accuracy = 0.507

Task 4
Training Accuracy = 0.5462222222222223
Validation Accuracy = 0.559
Testing Accuracy = 0.557

Task 5
Training Accuracy = 0.41355555555555557
Validation Accuracy = 0.409
Testing Accuracy = 0.414

Task 6
Training Accuracy = 0.49733333333333335
Validation Accuracy = 0.511
Testing Accuracy = 0.501

Task 7
Training Accuracy = 0.4865555555555556
Validation Accuracy = 0.488
Testing Accuracy = 0.488

Task 8
Training Accuracy = 0.5578888888888889


-----------------------
Epoch 50
Total Cost: 863684.2274169922

Task 1
Training Accuracy = 1.0
Validation Accuracy = 1.0
Testing Accuracy = 1.0

Task 2
Training Accuracy = 0.9766666666666667
Validation Accuracy = 0.965
Testing Accuracy = 0.97

Task 3
Training Accuracy = 0.8601111111111112
Validation Accuracy = 0.841
Testing Accuracy = 0.841

Task 4
Training Accuracy = 0.7495555555555555
Validation Accuracy = 0.762
Testing Accuracy = 0.744

Task 5
Training Accuracy = 0.4191111111111111
Validation Accuracy = 0.414
Testing Accuracy = 0.419

Task 6
Training Accuracy = 0.49733333333333335
Validation Accuracy = 0.512
Testing Accuracy = 0.503

Task 7
Training Accuracy = 0.4866666666666667
Validation Accuracy = 0.488
Testing Accuracy = 0.488

Task 8
Training Accuracy = 0.7247777777777777
Validation Accuracy = 0.75
Testing Accuracy = 0.705

Task 9
Training Accuracy = 0.6255555555555555
Validation Accuracy = 0.623
Testing Accuracy = 0.638

Task 10
Training Accuracy = 0.4731111111111111
Validatio

-----------------------
Epoch 100
Total Cost: 862458.0526123047

Task 1
Training Accuracy = 1.0
Validation Accuracy = 1.0
Testing Accuracy = 1.0

Task 2
Training Accuracy = 0.9981111111111111
Validation Accuracy = 0.978
Testing Accuracy = 0.984

Task 3
Training Accuracy = 0.8993333333333333
Validation Accuracy = 0.86
Testing Accuracy = 0.845

Task 4
Training Accuracy = 0.8064444444444444
Validation Accuracy = 0.8
Testing Accuracy = 0.791

Task 5
Training Accuracy = 0.4192222222222222
Validation Accuracy = 0.413
Testing Accuracy = 0.419

Task 6
Training Accuracy = 0.49733333333333335
Validation Accuracy = 0.512
Testing Accuracy = 0.503

Task 7
Training Accuracy = 0.4866666666666667
Validation Accuracy = 0.488
Testing Accuracy = 0.488

Task 8
Training Accuracy = 0.7266666666666667
Validation Accuracy = 0.751
Testing Accuracy = 0.708

Task 9
Training Accuracy = 0.6255555555555555
Validation Accuracy = 0.623
Testing Accuracy = 0.638

Task 10
Training Accuracy = 0.4731111111111111
Validatio

-----------------------
Epoch 150
Total Cost: 862216.5196533203

Task 1
Training Accuracy = 1.0
Validation Accuracy = 1.0
Testing Accuracy = 1.0

Task 2
Training Accuracy = 0.9982222222222222
Validation Accuracy = 0.981
Testing Accuracy = 0.981

Task 3
Training Accuracy = 0.9024444444444445
Validation Accuracy = 0.862
Testing Accuracy = 0.843

Task 4
Training Accuracy = 0.8206666666666667
Validation Accuracy = 0.812
Testing Accuracy = 0.809

Task 5
Training Accuracy = 0.4192222222222222
Validation Accuracy = 0.413
Testing Accuracy = 0.419

Task 6
Training Accuracy = 0.49733333333333335
Validation Accuracy = 0.512
Testing Accuracy = 0.503

Task 7
Training Accuracy = 0.4866666666666667
Validation Accuracy = 0.488
Testing Accuracy = 0.488

Task 8
Training Accuracy = 0.7268888888888889
Validation Accuracy = 0.752
Testing Accuracy = 0.709

Task 9
Training Accuracy = 0.6255555555555555
Validation Accuracy = 0.623
Testing Accuracy = 0.638

Task 10
Training Accuracy = 0.4731111111111111
Valida

In [ ]:
# View results in pd dataframe
import pandas as pd
from IPython.display import display
df = pd.DataFrame({
    'Training Accuracy': train_accs,
    'Validation Accuracy': val_accs,
    'Testing Accuracy': test_accs
    }, index=range(1, 21))
df.index.name = 'Task'
display(df)